<h1> Emoji Mining With SQL DB </h1>

In [142]:
from __future__ import division
import json
import tweepy
from tweepy.streaming import StreamListener
import pandas as pd
import re, collections
import numpy as np
import os
base_dir=os.path.expanduser('~')
import datetime
import time
import psycopg2
CONSUMER_KEY = 'JJNPMPuZubIrFIqDNARJRDPDb'
CONSUMER_SECRET = 'd6MlAqrJFcdyJac9aCQpBHPbwGs5eJB8zkTn5wA3tqBLHZgw0b'
OAUTH_TOKEN = '3229899732-piSMFy32Vi0VSyXJX8R9y2qrkr0piesoHXBdI3v'
OAUTH_TOKEN_SECRET = 'Jq9oTRMUjHRgA7NkJLLHIEyjtCRhYiFHdWkpBw28IBtHG'

In [141]:
import json

In [ ]:
Place(_api=<tweepy.api.API object at 0x109c5dd90>, country_code=u'AU', url=u'https://api.twitter.com/1.1/geo/id/01e4b0c84959d430.json', country=u'Australia', place_type=u'city', bounding_box=BoundingBox(_api=<tweepy.api.API object at 0x109c5dd90>, type=u'Polygon', coordinates=[[[148.995922, -35.48026], [148.995922, -35.147699], [149.263643, -35.147699], [149.263643, -35.48026]]]), full_name=u'Canberra, Australian Capital Territory', attributes={}, id=u'01e4b0c84959d430', name=u'Canberra')

In [137]:
#connect
conn = psycopg2.connect("dbname=emoji_db user=postgres password=darkmatter")
#on AWS: conn = psycopg2.connect("host=localhost port=5432 dbname=emoji_db user=postgres password=darkmatter")

#connect to AWS
#conn = psycopg2.connect("host=52.38.112.55 port=5432 dbname=emoji_db user=postgres password=darkmatter")
cur = conn.cursor()

In [6]:
cur.execute("SELECT date from emoji_tweet order by date DESC limit 1;")
last_date=cur.fetchone()
a=cur.execute("SELECT * from tweet_dump a join has_emoji b on a.id=b.tweet_id WHERE (a.date > %s and a.text != '' and b.has_emoji=True);",last_date)
a=cur.fetchall()
len(a)

0

In [122]:
cur.execute("SELECT tweet_id from emoji_tweet order by tweet_id DESC limit 1;")
last_id=cur.fetchone()
cur.execute("SELECT * from tweet_dump a left join has_emoji b on a.id=b.tweet_id WHERE ((a.id > %s) and a.text != '' and b.has_emoji=True) ;",last_id)
a=cur.fetchall()
len(a)

0

In [6]:
cur.execute("CREATE TABLE emoji_tweet (id serial PRIMARY KEY,\
    tweet_id integer,\
    date timestamp,\
    created_at timestamp,\
    text text,\
    retweet_count integer,\
    favorite_count integer,\
    lang varchar(5),\
    geo json,\
    coordinates json,\
    time_zone varchar(40),\
    name varchar(40),\
    user_name varchar(40),\
    emojiLabel text[],\
    emojiLabelFaceFilter bool[],\
    emojiCount integer[],\
    emojiCountSum integer,\
    emojiTypes integer,\
    prev_word text[],\
    next_word text[],\
    prev_sentence text[],\
    next_sentence text[],\
    mostFreqWord text,\
    mostFreqWordCount integer,\
    newlineCount integer,\
    emojiSkinLabel text[],\
    emojiSkinCount integer[],\
    emojiSkinCountSum integer,\
    emojiSkinTypes integer,\
    emojistrLabel text[],\
    emojistrCount integer[],\
    emojistrLen integer[],\
    emojistrTypes integer,\
    emojistr_prev_word text[],\
    emojistr_next_word text[],\
    emojistr_prev_sentence text[],\
    emojistr_next_sentence text[],\
    emojiPatternLabel text[],\
    emojiPatternCount integer[],\
    emojiPatternLen integer[],\
    emojiPatternTypes integer\
    );")
conn.commit() #submit change to db

In [9]:
cur.execute("CREATE TABLE tweet_dump (id serial PRIMARY KEY,\
    date timestamp,\
    created_at timestamp,\
    text text,\
    retweet_count integer,\
    favorite_count integer,\
    lang varchar(5),\
    geo json,\
    coordinates json,\
    time_zone varchar(40),\
    name varchar(40),\
    user_name varchar(40)\
    );")
conn.commit() #submit change to db

In [10]:
cur.execute("CREATE TABLE has_emoji (id serial PRIMARY KEY,\
    tweet_id integer,\
    has_emoji bool\
    );")
conn.commit() #submit change to db

In [139]:
#create index result DB
#word,face_filter,pattern_type,user_lang,xdata,ydata

cur.execute("CREATE TABLE emoji_search (id serial PRIMARY KEY,\
    date timestamp,\
    searchTerm text,\
    emojiLabel text[],\
    emojiCount integer[],\
    FreqFilter text,\
    FaceFilter text,\
    PatternType text,\
    Lang text,\
    dateRange timestamp[]\
    );")
conn.commit() #submit change to db

In [140]:
#create index result DB
#word,user_lang,xdata,ydata

cur.execute("CREATE TABLE emoji_skin_search (id serial PRIMARY KEY,\
    date timestamp,\
    searchTerm text,\
    emojiLabel text[],\
    emojiCount integer[],\
    Lang text,\
    dateRange timestamp[]\
    );")
conn.commit() #submit change to db

In [ ]:
#Map
cur.execute("CREATE TABLE emoji_map (id serial PRIMARY KEY,\
    date timestamp,\
    searchTerm text,\
    timezones text[],\
    topemojiLabel text[],\
    topemojiCount integer[],\
    FreqFilter text,\
    FaceFilter text,\
    PatternType text,\
    Lang text,\
    dateRange timestamp[]\
    );")
conn.commit() #submit change to db

In [640]:
cur.execute("SELECT text from emoji_tweet WHERE text LIKE '%🍆%';")
result=cur.fetchall()

In [641]:
[_u(text[0]) for text in result]

[u'RT @NikoOrzuza: Basta de esperar cosas que no van a pasar \U0001f645\u270b',
 u"Voto por 'Te dar\xe9 mi coraz\xf3n' de @AbrahamMateoMus Es una balada preciosa! \U0001f60d\U0001f495 Merece ser N1 \u270b #N1CanalFiesta12 ejsjsnd",
 u'Buenas noches!.. \U0001f319\U0001f60a\U0001f60d\U0001f48b\u270b',
 u'@Y_M_kis317 5\u6b73\u3088\uff1f\uff08\u7206\u7b11\uff09\n\n\u307e\u3060\u304b\u308f\u3044\u3044\u9854\u3057\u3068\u3063\u305f\u3088 \u270b\U0001f3fb',
 u"\u0648\u0639\u062f \u061b \u270b\n\u0645\u0627 \u0623\u0639\u0627\u062a\u0628\u0643 ..\n\u0628\u062e\u0641\u064a \u0628\u0640 \u0635\u062f\u0631\u064a \u0636\u064a\u0642\u062a\u064a \u060c\n\u0648\u0623\u062c\u0627\u0645\u0644\u0643 \u060c\n\n\u0628\u0645\u0634\u064a \u0645\u0639\u0643 \u0645\u062b\u0644 \u0627\u0644\u063a\u0631\u064a\u0628\n\u0628\u0640 \u0623\u062d\u0646 \u0644\u0643 \u061f\n\u0648\u0623\u0634\u062a\u0627\u0642 \u0644\u0643 \u061b\n\u0644\u0643\u0646 \u0648\u0639\u062f \u0645\u0627 \u0623\u0639\u0644\u0645\u0643 ') !"

<h1> Tweepy to postgreSQL</h1>

In [ ]:
#emj_codes have the skin codes removed. not counted

In [168]:
[key for key in emoji_key]

[u'Code', u'Unicode', u'Name', u'Annotations', u'Action']

In [676]:
##include one like u'\u270d', second line
can_have_skin=[key[0:2] for key in emj_codes if re.findall(emj_codes_skin[0],key) != []]
can_have_skin += [key[0:1] for key in emj_codes if len(key[0:2].encode("utf-8"))==6 and re.findall(emj_codes_skin[0],key) != []]

In [678]:
#use this as the label
u'🖐'

u'\U0001f590'

In [677]:
can_have_skin

[u'\U0001f466',
 u'\U0001f467',
 u'\U0001f468',
 u'\U0001f469',
 u'\U0001f474',
 u'\U0001f475',
 u'\U0001f476',
 u'\U0001f471',
 u'\U0001f46e',
 u'\U0001f472',
 u'\U0001f473',
 u'\U0001f477',
 u'\U0001f478',
 u'\U0001f482',
 u'\U0001f575',
 u'\U0001f385',
 u'\U0001f47c',
 u'\U0001f486',
 u'\U0001f487',
 u'\U0001f470',
 u'\U0001f64d',
 u'\U0001f64e',
 u'\U0001f645',
 u'\U0001f646',
 u'\U0001f481',
 u'\U0001f64b',
 u'\U0001f647',
 u'\U0001f64c',
 u'\U0001f64f',
 u'\U0001f6b6',
 u'\U0001f3c3',
 u'\U0001f483',
 u'\U0001f4aa',
 u'\U0001f448',
 u'\U0001f449',
 u'\u261d\ufe0f',
 u'\U0001f446',
 u'\U0001f595',
 u'\U0001f447',
 u'\u270c\ufe0f',
 u'\U0001f596',
 u'\U0001f918',
 u'\U0001f590',
 u'\u270b\ud83c',
 u'\U0001f44c',
 u'\U0001f44d',
 u'\U0001f44e',
 u'\u270a\ud83c',
 u'\U0001f44a',
 u'\U0001f44b',
 u'\U0001f44f',
 u'\U0001f450',
 u'\u270d\ud83c',
 u'\U0001f485',
 u'\U0001f442',
 u'\U0001f443',
 u'\U0001f6a3',
 u'\U0001f6c0',
 u'\U0001f3c4',
 u'\U0001f3ca',
 u'\u26f9\ud83c',
 u'\U0001f3c

In [298]:
#read emoji codes:
emoji_key = pd.read_excel(base_dir+'/emojify/data/emoji_list.xlsx', encoding='utf-8', index_col=0, skiprows=1)
emj_codes_skin=[code for code,name in zip(emoji_key['Unicode'],emoji_key['Name']) if ('FITZPATRICK' in name)]
emj_codes=[code for code in emoji_key['Unicode'] if code!="Browser" \
           if (code not in emj_codes_skin) if sum([c=="*" for c in code])==0]
##yellow, potential for skin tone
can_have_skin=[key[0:2] for key in emj_codes if re.findall(emj_codes_skin[0],key) != []]
can_have_skin += [key[0:1] for key in emj_codes if len(key[0:2].encode("utf-8"))==6 and re.findall(emj_codes_skin[0],key) != []]
#remove common face emojis
face_index=range(69)
emj_codes_face=[code for index,code in zip(emoji_key.index,emoji_key['Unicode']) if index in face_index]

_u = lambda t: t.decode('UTF-8', 'replace') if isinstance(t, str) else t

def get_keys(tweet):
    max_index=0
    max_count=len(tweet['statuses'][max_index].keys())
    
    for ii in range(len(tweet['statuses'])):
        if max_count < len(tweet['statuses'][ii].keys()):
            max_index=ii
            
    return tweet['statuses'][max_index].keys()

def print_tweets_w_emj(T_DF,num=0):
    for item in T_DF['text'][T_DF['emjSum']>=num]:
        print(item)
        
def make_twitter_DF(tweet): #create PD dataframe
    columns=get_keys(tweet)
    for ii in range(len(tweet['statuses'])): 
        diff= list(set(columns)-set(tweet['statuses'][ii].keys()))
        for val in diff:
            tweet['statuses'][ii][val]=""
    return pd.DataFrame([tweet['statuses'][ii] for ii in range(len(tweet['statuses']))], columns=get_keys(tweet)) 

def Search_Word(q):
    # Initiate the connection to Twitter REST API
    twitter = Twitter(auth=OAuth( OAUTH_TOKEN, OAUTH_TOKEN_SECRET, CONSUMER_KEY,  CONSUMER_SECRET))       
    tweet= twitter.search.tweets(q=q,lang='en', result_type='recent',count=200)
    T_DF=make_twitter_DF(tweet)
    
    #long search function, more than 100
    #T_DF=long_search(twitter,q,300)
    
    T_DF['emjText']=[[(emcode, len(re.findall(emcode,T_DF['text'][ii]))) for emcode in emj_codes\
                      if (len(re.findall(emcode,T_DF['text'][ii])) > 0)] for ii in range(len(T_DF))]
    T_DF['emjSum']=[sum([item[1] for item in T_DF['emjText'][ii]]) for ii in range(len(T_DF))]
    T_DF['emjTypes']=[len(T_DF['emjText'][ii]) for ii in range(len(T_DF))]
    T_DF['searchWordSum']=[len(re.findall(q,T_DF['text'][ii].lower())) for ii in range(len(T_DF))]
    return T_DF

def Analyze_Freq(T_DF,emj_per_tweet=1,rep_emj_per_tweet=1, emj_types_per_tweet=1, nword=0):
    DFcut=T_DF[(T_DF['emjSum']>=emj_per_tweet) & (T_DF['searchWordSum']>=nword) \
               & (T_DF['emjTypes']>=emj_types_per_tweet)] 
    
    S=dict()
    for key, value in sum(DFcut['emjText'],[]):
        if value>=rep_emj_per_tweet:
            try:
                S[key] +=value
            except KeyError:
                S[key]=value
        
    return pd.DataFrame([(key,S[key]) for key in S],columns=('emoji','freq'))

def count_words(text):
    S = collections.defaultdict(lambda:0)
    for word in text.split():
        S[word.lower()]+=1
    return max(S, key=S.get), S[max(S, key=S.get)]

def emoji_split_all(text):
    '''add a space before and after the emoji, then remove double spaces. Keep \n'''
    lines=[]
    for line in text.split('\n'):
        for emcode in emj_codes:
            if (len(re.findall(emcode,line))) > 0:
                line=line.replace(emcode,' '+emcode+' ')
        for skin in emj_codes_skin:
            line=_u(line)
            line=line.replace(' '+skin,skin+' ') #put the skin codes back into place but add a space afterwards
        lines.append(' '.join(line.split()))
    return '\n'.join(lines)

def emoji_split(text):
    '''add a space before and after the emoji. Do not add space between consecutive emojis unless seperated by a space.
    Remove double spaces. Keep \n'''
    lines=[]
    for line in text.split('\n'):
        line=_u(line)
        for emcode in emj_codes:
            if (len(re.findall(emcode,line))) > 0:
                line=line.replace(emcode,'^'+emcode+'^')
        for skin in emj_codes_skin:
            line=_u(line)
            line=line.replace('^'+skin,skin+'^') #put the skin codes back into place but add a space afterwards
        line=line.replace('^^','')
        line=line.replace('^',' ')
        lines.append(' '.join(line.split()))
    return '\n'.join(lines)

def emoji_split_line(line):
    '''Add a space before and after the emoji then remove double spaces, while keeping original spaces between emojis'''
    line=line.replace(' ','^')
    for emcode in emj_codes:
        if (len(re.findall(emcode,line))) > 0:
            line=line.replace(emcode,' '+emcode+' ')
    for skin in emj_codes_skin:
        line=_u(line)
        line=line.replace(' '+skin,skin+' ') #put the skin codes back into place but add a space afterwards
    return ' '.join(line.split())

def NextWord(text,emj):
    try:
        last_index=len(text)-(text[::-1].index(emj[::-1]))-2
    except ValueError:
        last_index=len(text)
    return ''.join(text[last_index+2:].split()[:1])

def surroundingText(text,emojiLabel):
    #work with text after running the split_all function, seach leabels with surrunding space for index
    #We search with ' emcode ' then for the index do ' emcode '.strip() to remove white space
    emojiLabel=[' '+label+' ' for label in emojiLabel] #add space
    return np.array([(emcode.strip(), ''.join(text[:text.index(emcode)].split()[-1:]),\
                    NextWord(text,emcode),\
                    ' '.join(text[:text.index(emcode)].split()[-7:]),\
                    ' '.join(text[text.index(emcode)+len(emcode):].split()[:7])) for emcode in emojiLabel\
                      if (re.findall(emcode,text) !=[]) ])
    
####################################################################################
def write_emoji_usage(tweet,has_emoji):
    entry = {"date": datetime.datetime.utcnow(),\
             "created_at": tweet.created_at,\
             "retweet_count": tweet.retweet_count,\
             "favorite_count": tweet.favorite_count,\
             "lang": tweet.lang,\
             "geo": tweet.geo,\
             "coordinates": tweet.coordinates,\
             "time_zone": tweet.user.time_zone,\
             "name":tweet.user.name, "user_name":tweet.user.screen_name,\
             "has_emoji":has_emoji}
    emoji_usage.insert_one(entry).inserted_id

def checkNone(val):
    return val if val else ''
def checkNoneJSON(val):
    return json.dumps(val) if val else '{}'

def getMongoTweet(tweet):
    #tweet data:
    tweet_id = 0
    date=tweet['date']
    created_at=tweet['created_at']
    original_text=tweet['text']
    retweet_count=tweet['retweet_count']
    favorite_count=tweet['favorite_count']
    lang=tweet['lang']
    geo=checkNoneJSON(tweet['goe'])
    coordinates=checkNoneJSON(tweet['coordinates'])
    try:
        time_zone=tweet['time_zone']
    except KeyError:
        time_zone=''
    try:
        name=tweet['name']
    except KeyError:
        name=''
    try:
        user_name=tweet['user_name']
    except KeyError:
        user_name=''
    
    return tweet_id,date,created_at,original_text,retweet_count,favorite_count,lang,geo,coordinates,\
    time_zone,name,user_name
    
#class MineEmojis:    
def analyze_tweet_emojis(SQL_return,Mongo=False):
    #tweet data:
    has_emoji=False
    
    if Mongo:
        tweet_id,date,created_at,original_text,retweet_count,favorite_count,lang,geo,coordinates,\
        time_zone,name,user_name=getMongoTweet(SQL_return)
    else:
        tweet_id = SQL_return[0]
        date=SQL_return[1]
        created_at=SQL_return[2]
        original_text=SQL_return[3]
        retweet_count=SQL_return[4]
        favorite_count=SQL_return[5]
        lang=SQL_return[6]
        geo=checkNoneJSON(SQL_return[7])
        coordinates=checkNoneJSON(SQL_return[8])
        time_zone=SQL_return[9]
        name=SQL_return[10]
        user_name=SQL_return[11]
    
    text=emoji_split(original_text)
    emjText=np.array([(emcode, len(re.findall(emcode,text))) for emcode in emj_codes\
                      if (len(re.findall(emcode,text)) > 0)])

    if len(emjText) >0:
        print(text)
        has_emoji=True
        mostFreqWord, mostFreqWordCount = count_words(text)
        newlineCount= text.count('\n')
        #create arrays to save in SQL. Sorted by frequency
        emojiLabel=emjText[np.argsort(emjText[:, 1])[::-1]][:,0]
        emojiLabelFaceFilter= np.in1d(emojiLabel,emj_codes_face,invert=True)
        emojiCount=np.array(emjText[np.argsort(emjText[:, 1])[::-1]][:,1], dtype=int)
        emojiTypes=len(emojiCount)
        emojiCountSum=sum(emojiCount)
        surrounding_text=surroundingText(text,emojiLabel) #sorted by frequency
        prev_word=surrounding_text[:,1]
        next_word=surrounding_text[:,2]
        prev_sentence=surrounding_text[:,3]
        next_sentence=surrounding_text[:,4]

        #build array of emoji strings
        emj_str = np.array([(emj_str, int(len(emj_str)/2)) for emj_str in sum([''.join([word if word in emj_codes+[' '] \
        else 'T' for word in emoji_split_line(line).split()]).rsplit('T') for line in text.split('\n')],[]) if emj_str != ''])
        
        #analyze emoji strings, cut away length 1 emojis and call new array a:
        if len(emj_str)==0:
            emojistrLabel,emojistrCount,emojistrLen,emojistrTypes,emojistr_prev_word,emojistr_next_word,\
            emojistr_prev_sentence,emojistr_next_sentence,emojiPatternLabel,emojiPatternCount,emojiPatternLen,emojiPatternTypes=\
            [],[],[],0,[],[],[],[],[],[],[],0
        else: #try to find strings, but first filter length 1 and those with length 2(with skin codes)
            d=collections.defaultdict(lambda:0)
            for key in emj_str[:,0]:
                d[key]+=1
            a=np.array([d.values(),d.keys(),[int(len(key)/2) for key in d.keys()]])
            #remove single emojis and double if skin code is included:
            skin_cut=~np.bool8(((np.int32(a[2,:]))==2) & (np.array([sum([len(re.findall(emcode,val)) for emcode in emj_codes_skin]) for val in a[1,:]])))
            multi_cut=(np.int32(a[2,:])>1) & skin_cut
            a=a[:,multi_cut]

            if len(a[0])==0:
                emojistrLabel,emojistrCount,emojistrLen,emojistrTypes,emojistr_prev_word,emojistr_next_word,\
                emojistr_prev_sentence,emojistr_next_sentence,emojiPatternLabel,emojiPatternCount,emojiPatternLen,\
                emojiPatternTypes=\
                [],[],[],0,[],[],[],[],[],[],[],0

            else:
                sort_index=a.argsort(axis=1)
                emojistrLabel=a[[1,sort_index[0]]][::-1]
                emojistrCount=np.array(a[[0,sort_index[0]]][::-1],dtype=int)
                emojistrLen=np.array(a[[2,sort_index[0]]][::-1],dtype=int)
                emojistrTypes=len(emojistrCount)
                #add emjStr CountSum
                surrounding_str_text=surroundingText(text,emojistrLabel) #sorted by frequency
                emojistr_prev_word=surrounding_str_text[:,1]
                emojistr_next_word=surrounding_str_text[:,2]
                emojistr_prev_sentence=surrounding_str_text[:,3]
                emojistr_next_sentence=surrounding_str_text[:,4]
                #find emoji str patterns
                pattern=np.array([(emcode, len(re.findall(emcode,text))) for emcode in emojistrLabel])
                emojiPatternLabel=pattern[np.argsort(pattern[:, 1])[::-1]][:,0] 
                emojiPatternCount=np.array(pattern[np.argsort(pattern[:, 1])[::-1]][:,1],dtype=int)
                emojiPatternLen=np.array([np.int32(len(val)/2) for val in emojiPatternLabel],dtype=int)
                emojiPatternTypes=len(emojiPatternCount)

        #skin tone information
        emjText_skin=np.array([(emcode, len(re.findall(emcode,text))) for emcode in emj_codes_skin\
                      if (len(re.findall(emcode,text)) > 0)])
        if len(emjText_skin)==0:
            emojiSkinLabel, emojiSkinCount,emojiSkinCountSum,emojiSkinTypes= [],[],0,0
        else:
            #create arrays to save in SQL. Sorted by frequency
            emojiSkinLabel=emjText_skin[np.argsort(emjText_skin[:, 1])[::-1]][:,0] 
            emojiSkinCount=np.array(emjText_skin[np.argsort(emjText_skin[:, 1])[::-1]][:,1],dtype=int)
            emojiSkinCountSum=sum(emojiSkinCount)
            emojiSkinTypes=len(emojiSkinCount)
        
        insertIntoSQL(tweet_id, date,created_at,text,retweet_count,favorite_count,lang,geo,coordinates,time_zone,name,user_name,\
    emojiLabel,emojiLabelFaceFilter,emojiCount,emojiCountSum,emojiTypes,prev_word,next_word,prev_sentence,next_sentence,mostFreqWord,\
    mostFreqWordCount,newlineCount,emojiSkinLabel,emojiSkinCount,emojiSkinCountSum,emojiSkinTypes,emojistrLabel,\
    emojistrCount,emojistrLen,emojistrTypes,emojistr_prev_word,emojistr_next_word,emojistr_prev_sentence,\
    emojistr_next_sentence,emojiPatternLabel,emojiPatternCount,emojiPatternLen,emojiPatternTypes)
    
    if not Mongo:
        has_emoji_SQL(tweet_id, has_emoji)
    
#class MineEmojis:    
def mine_tweets(tweet):
    print(tweet.text)
    has_emoji=True
    #tweet data:
    date= datetime.datetime.utcnow()
    created_at = tweet.created_at
    text = tweet.text
    retweet_count = tweet.retweet_count
    favorite_count = tweet.favorite_count
    lang=checkNone(tweet.lang)
    geo = checkNoneJSON(tweet.geo)
    time_zone = checkNone(tweet.user.time_zone)
    coordinates = checkNoneJSON(tweet.coordinates)
    name = checkNone(tweet.user.name)
    user_name = checkNone(tweet.user.screen_name)
    dumpIntoSQL(date,created_at,text,retweet_count,favorite_count,lang,geo,coordinates,time_zone,name,user_name)

def dumpIntoSQL(date,created_at,text,retweet_count,favorite_count,lang,geo,coordinates,time_zone,name,user_name):
    cur.execute("INSERT INTO tweet_dump (\
    date,\
    created_at,\
    text,\
    retweet_count,\
    favorite_count,\
    lang,\
    geo,\
    coordinates,\
    time_zone,\
    name,\
    user_name\
    )\
    VALUES (\
    %s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s\
    )",(\
    date,\
    created_at,\
    text,\
    retweet_count,\
    favorite_count,\
    lang,\
    geo,\
    coordinates,\
    time_zone,\
    name,\
    user_name\
    ))
    conn.commit() #submit change to db

def has_emoji_SQL(tweet_id, has_emoji):
    cur.execute("INSERT INTO has_emoji (\
    tweet_id,\
    has_emoji\
    )\
    VALUES (\
    %s,%s\
    )",(\
    tweet_id,\
    has_emoji\
    ))
    conn.commit() #submit change to db
    
def insertIntoSQL(tweet_id, date,created_at,text,retweet_count,favorite_count,lang,geo,coordinates,time_zone,name,user_name,\
    emojiLabel,emojiLabelFaceFilter,emojiCount,emojiCountSum,emojiTypes,prev_word,next_word,prev_sentence,next_sentence,mostFreqWord,\
    mostFreqWordCount,newlineCount,emojiSkinLabel,emojiSkinCount,emojiSkinCountSum,emojiSkinTypes,emojistrLabel,\
    emojistrCount,emojistrLen,emojistrTypes,emojistr_prev_word,emojistr_next_word,emojistr_prev_sentence,\
    emojistr_next_sentence,emojiPatternLabel,emojiPatternCount,emojiPatternLen,emojiPatternTypes):
    cur.execute("INSERT INTO emoji_tweet (\
    tweet_id,\
    date,\
    created_at,\
    text,\
    retweet_count,\
    favorite_count,\
    lang,\
    geo,\
    coordinates,\
    time_zone,\
    name,\
    user_name,\
    emojiLabel,\
    emojiLabelFaceFilter,\
    emojiCount,\
    emojiCountSum,\
    emojiTypes,\
    prev_word,\
    next_word,\
    prev_sentence,\
    next_sentence,\
    mostFreqWord,\
    mostFreqWordCount,\
    newlineCount,\
    emojiSkinLabel,\
    emojiSkinCount,\
    emojiSkinCountSum,\
    emojiSkinTypes,\
    emojistrLabel,\
    emojistrCount,\
    emojistrLen,\
    emojistrTypes,\
    emojistr_prev_word,\
    emojistr_next_word,\
    emojistr_prev_sentence,\
    emojistr_next_sentence,\
    emojiPatternLabel,\
    emojiPatternCount,\
    emojiPatternLen,\
    emojiPatternTypes\
    )\
    VALUES (\
    %s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s\
    )",(\
    tweet_id,\
    date,\
    created_at,\
    text,\
    retweet_count,\
    favorite_count,\
    lang,\
    geo,\
    coordinates,\
    time_zone,\
    name,\
    user_name,\
    list(emojiLabel),\
    list(emojiLabelFaceFilter),\
    list(emojiCount),\
    emojiCountSum,\
    emojiTypes,\
    list(prev_word),\
    list(next_word),\
    list(prev_sentence),\
    list(next_sentence),\
    mostFreqWord,\
    mostFreqWordCount,\
    newlineCount,\
    list(emojiSkinLabel),\
    list(emojiSkinCount),\
    emojiSkinCountSum,\
    emojiSkinTypes,\
    list(emojistrLabel),\
    list(emojistrCount),\
    list(emojistrLen),\
    emojistrTypes,\
    list(emojistr_prev_word),\
    list(emojistr_next_word),\
    list(emojistr_prev_sentence),\
    list(emojistr_next_sentence),\
    list(emojiPatternLabel),\
    list(emojiPatternCount),\
    list(emojiPatternLen),\
    emojiPatternTypes\
    ))
    conn.commit() #submit change to db
        
class StdOutListener(StreamListener):
    """ A listener handles tweets that are received from the stream.
    This is a basic listener that just prints received tweets to stdout.
    """
    #Mine=MineEmojis()
    def on_status(self, status):
        mine_tweets(status)
        return True
    def on_error(self, status):
        print(status)

In [ ]:
#run the streamer
l=StdOutListener()
auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(OAUTH_TOKEN, OAUTH_TOKEN_SECRET)
api = tweepy.API(auth, wait_on_rate_limit=True,wait_on_rate_limit_notify=True)
myStream = tweepy.Stream(auth = api.auth, listener=l)
myStream.sample()

😂😂😂
RT @67ertg1: #토토
#토토

사-설-토-토-추-천

🐽🅾🐽  https://t.co/4HZ4pKAS6S  🐽🅾🐽

네-💫
임-💫
드-💫
사-💫
다-💫
리-💫

💐💐크로스,축배팅,단폴더 배팅 가능💐💐
RT @thehill: Susan Sarandon: I've faced "so much shaming" for supporting Sanders https://t.co/Y3gcfrSt7T https://t.co/bCPx2hE1uJ
RT @MrKrabsBIurMeme: When you reach in your pocket and don't feel your phone. https://t.co/1kPuUWc06H
RT @brklnbound: de verdad que a veces me siento más idiota que el tío que tradujo el nombre de Homer al español como Homero
RT @myamarissa_: I want you.
I've tried too ignore it everytime u phone, but i never come close X
تفاعل الناس و دفاعهم عن حقوق بعض شيء يبعث على التفاءل 🙏🏼 #ازالة_قرية_عمق_جنوب_مكه
RT @ProfesorMorboso: https://t.co/DhkxA9J9Qx
Good morning Asia !
RT @JoeyGraceffa: ⚡️ GLOW IN THE DARK HAIR DIY! ⚡ ️https://t.co/xtX86K31sI RT/LIKE FOR A SURPRISE! ⚡️ https://t.co/tR5uCvrHsF
RT @salahabdallah8: أقسم بالله أننى أشجع الأهلى أمام أي فريق غير مصرى وكأننى أهلاوي صميم..أهلاوي إبن أهلاويه..مش زي فيلم الزمهلاويه 🌹❤️🌹 ht…
El dolor de 

In [33]:
cur.execute("SELECT id from emoji_tweet order by id DESC limit 1;")
last_analyzed_id=cur.fetchone()

In [65]:
#analyze the emojis
#find the last id analyzed in the emoji_tweet table
cur.execute("SELECT tweet_id from emoji_tweet order by tweet_id DESC limit 1;")
try:
    last_analyzed_id=cur.fetchone()[0]
except TypeError:
    last_analyzed_id=0
    
for ID in range(last_analyzed_id,100000000):
    cur.execute("SELECT * from tweet_dump where id = {:d};".format(ID+1))
    SQL_result=cur.fetchone()
    analyze_tweet_emojis(SQL_result)
    
    

💛💭💛💭💛
عن حُزنك ؟
هناك صلاه .

عن ضيقتك المفاجئة ؟
هناك قرآن .

عن أمنياتك المستقبليه ؟
هناك دعاء .

عن النعيم الأبدي ؟
هناك جنّه .

💛💭💛💭💛
RT @NiallOfficial: Hello lovely people . . How are we all today? 😍😊😎
I love my best friends 💗
RT @laurxnf: there are a million important things to do but none as important as be here next to you 💫 . https://t.co/wKklFs8I1i
#المملكه_والفولورز_وهواك
و ربنا اللى ما هيعمل فولو لعلقو لما يبانلو صاحب 😂😂😂😂 https://t.co/pgEt5Ol4FW
RT @LForestermjb: #사설사다리
#스포츠토토사이트추천

💫 네-임-드-사-다-리 🌲
👩 먹튀없음(명품놀이터)♠
👨 클릭 https://t.co/6X6Spwisey 추천인코드:luck 👷

🌹👩 JOIN NOW 🔒👂
🇶
♣
망롣닝랖뚜 https://t…
RT @StefanKaroli: dope"@XtiandelaFTrain: Last gain tweet 😊

Follow everyone that retweet this to gain 100 🆕 follows tonight ✌

#GainWithXti…
RT @deadpoooI: Boys in Red Jackets 🔥👌 https://t.co/uUB80sOnxq
RT @SoAmIWrong: LAS ABUELAS SON LAS MEJORES. 💝👼
RT @mariobautista_: No hay nada más bonito que ver a gente sonriendo por la vida 😍🙌🏻
All imma see tomorrow is niggas smoking all down my s

KeyboardInterrupt: 

In [62]:
_u = lambda t: t.decode('UTF-8', 'replace') if isinstance(t, str) else t

In [73]:
word=_u(word)
word.encode('utf-8')

'\xf0\x9f\x8d\x9f'

In [70]:
word="🍟"
word=word.lower()
word=word.replace("'","''")
cur.execute("SELECT T.Label,SUM(T.Freq) as TFreq From (SELECT unnest(emojiLabel) as Label, unnest(emojiCount) as Freq FROM emoji_tweet WHERE (LOWER(text) LIKE '%🍟%')) as T group by T.Label order by TFreq DESC limit 15;".format(word))

<h1> Search SQL DB </h1>

In [49]:
cur.execute("SELECT emojiLabel,emojiCount FROM emoji_tweet where LOWER(text) LIKE '%🍟%';")
result=cur.fetchall()
#for vals in cur.fetchall():
#    for val in vals:
#        print(val)

In [66]:
start=datetime.datetime.utcnow()
cur.execute("SELECT emojiLabel,emojiCount FROM emoji_tweet where LOWER(text) LIKE '% %';")
result=cur.fetchall()

S=collections.defaultdict(lambda:0)
for tup in result:
    for key,val in zip(tup[0],tup[1]):
        S[key]+=val
datetime.datetime.utcnow()-start

datetime.timedelta(0, 3, 811913)

In [24]:
cur.execute("SELECT T.Label,SUM(T.Freq) as TFreq From (SELECT unnest(emojiLabel) as Label, unnest(emojiCount) as Freq, unnest(emojiLabelFaceFilter) as FF FROM emoji_tweet WHERE (LOWER(text) LIKE '%{:s}%')) as T WHERE(T.FF is True) group by T.Label order by TFreq DESC limit 15;".format("Trump".lower()))
result=cur.fetchall()
result

[('\xf0\x9f\x87\xba\xf0\x9f\x87\xb8', 591L),
 ('\xf0\x9f\x92\x80', 200L),
 ('\xf0\x9f\x9a\xa8', 146L),
 ('\xf0\x9f\x94\xb4', 115L),
 ('\xe2\x80\xbc\xef\xb8\x8f', 113L),
 ('\xf0\x9f\x9a\x82', 100L),
 ('\xf0\x9f\x96\x95', 100L),
 ('\xf0\x9f\x94\xa5', 94L),
 ('\xf0\x9f\x96\x95\xf0\x9f\x8f\xbd', 91L),
 ('\xe2\x9c\x94\xef\xb8\x8f', 80L),
 ('\xf0\x9f\x8c\x80', 56L),
 ('\xf0\x9f\x99\x8f', 54L),
 ('\xf0\x9f\x91\x8d', 52L),
 ('\xf0\x9f\x94\xb9', 48L),
 ('\xf0\x9f\x94\xb5', 45L)]

In [30]:
xdata=[val[0] for val in result]
ydata=[val[1] for val in result]

[591L,
 200L,
 146L,
 115L,
 113L,
 100L,
 100L,
 94L,
 91L,
 80L,
 56L,
 54L,
 52L,
 48L,
 45L]

In [44]:
cur.execute("SELECT text from emoji_tweet WHERE (emojiCountSum > 30) order by random() limit 40;")

In [45]:
result=[text[0] for text in cur.fetchall()]

<h1> Do the same in postgrSQL in 1/2 the time </h1>

In [ ]:
#most freq emoji, count all:
"SELECT T.Label,SUM(T.Freq) as TFreq From (SELECT unnest(emojiLabel) as Label, unnest(emojiCount) as Freq FROM emoji_tweet WHERE (LOWER(text) LIKE '% %')) as T group by T.Label order by TFreq DESC limit 15;"
#with the face filter:
"SELECT T.Label,SUM(T.Freq) as TFreq From (SELECT unnest(emojiLabel) as Label, unnest(emojiCount) as Freq, unnest(emojiLabelFaceFilter) as FF FROM emoji_tweet WHERE (LOWER(text) LIKE '% %')) as T WHERE(T.FF is True) group by T.Label order by TFreq DESC limit 15;"

#select emoji str
"SELECT T.Label,SUM(T.Freq) as TFreq From (SELECT unnest(emojistrLabel) as Label, unnest(emojistrCount) as Freq FROM emoji_tweet WHERE (LOWER(text) LIKE '% %')) as T group by T.Label order by TFreq DESC limit 15;"
#with FaceFilter on emojiLabel:
"SELECT T.Label,SUM(T.Freq) as TFreq From (SELECT unnest(emojistrLabel) as Label, unnest(emojistrCount) as Freq, unnest(emojiLabelFaceFilter) as FF FROM emoji_tweet WHERE (LOWER(text) LIKE '% %')) as T WHERE(FF is True) group by T.Label order by TFreq DESC limit 15;"
#cut all tweets with a face: #the cut happens after groupy so it's not quite accurate
"SELECT T.Label,SUM(T.Freq) as TFreq,bool_and(T.FF) as prodFF From (SELECT unnest(emojistrLabel) as Label, unnest(emojistrCount) as Freq, unnest(emojiLabelFaceFilter) as FF FROM emoji_tweet WHERE (LOWER(text) LIKE '% %')) as T group by T.Label  HAVING(bool_and(FF) is True) order by TFreq DESC limit 15;"
#cut all tweets with face... should be faster, using: NOT (emojiLabelFaceFilter @> ARRAY[False])
"SELECT T.Label,SUM(T.Freq) as TFreq From (SELECT unnest(emojistrLabel) as Label, unnest(emojistrCount) as Freq FROM emoji_tweet WHERE (emojistrTypes>0 AND LOWER(text) LIKE '% %' and NOT(emojiLabelFaceFilter @> ARRAY[False]))) as T group by T.Label order by TFreq DESC limit 15;"


#select emojiPattern
"SELECT T.Label,SUM(T.Freq) as TFreq From (SELECT unnest(emojiPatternLabel) as Label, unnest(emojiPatternCount) as Freq FROM emoji_tweet WHERE (LOWER(text) LIKE '% %')) as T group by T.Label order by TFreq DESC limit 15;"

#Select ALL
"SELECT T.Label,SUM(T.Freq) as TFreq From (SELECT unnest(array_cat(emojiLabel,array_cat(emojistrLabel,emojiPatternLabel))) as Label, unnest(array_cat(emojiCount,array_cat(emojistrCount,emojiPatternCount))) as Freq FROM emoji_tweet WHERE (LOWER(text) LIKE '%dog%')) as T group by T.Label order by TFreq DESC limit 15;"

<h1> Search for most freqent emojiLabel or emoji string Label per tweet</h1>

In [ ]:
#emojiLabel
"SELECT emojistrLabel[1],SUM(emojiStrCount[1]) as Freq FROM emoji_tweet where LOWER(text) LIKE '% %' group by emojistrLabel[1] order by Freq DESC limit 15;"
#select emojiLabel with face filter
"SELECT emojiLabel[1],SUM(emojiCount[1]) as Freq FROM emoji_tweet a where( LOWER(text) LIKE '% %' and a.emojiLabelFaceFilter[1] is true) group by emojiLabel[1] order by Freq DESC limit 15;"

#emojiStrLabel
"SELECT emojistrLabel[1] as label,SUM(emojiStrCount[1]) as Freq FROM emoji_tweet WHERE(emojiStrTypes>0 AND LOWER(text) LIKE '%trump%') group by label order by Freq DESC limit 15;"
#with face filter (on emojiLabel) ... might not be complete
"SELECT emojistrLabel[1],SUM(emojistrCount[1]) as Freq FROM emoji_tweet a WHERE(emojiStrCount[1]>0 AND LOWER(text) LIKE '% %' and a.emojiLabelFaceFilter[1] is true) group by emojistrLabel[1] order by Freq DESC limit 15;"

#emojiPattern:
"SELECT emojiPatternLabel[1],SUM(emojiPatternCount[1]) as Freq FROM emoji_tweet a where( LOWER(text) LIKE '% %') group by emojiPatternLabel[1] order by Freq DESC limit 15;"
#with face filter
"SELECT emojiPatternLabel[1],SUM(emojiPatternCount[1]) as Freq FROM emoji_tweet a where( LOWER(text) LIKE '% %' and a.emojiLabelFaceFilter[1] is true) group by emojiPatternLabel[1] order by Freq DESC limit 15;"

In [ ]:
🏼🏻🏾🏽🏿

<h1> Search for skin tone info </h1>

In [ ]:
"SELECT T.Label,SUM(T.Freq) as TFreq From (SELECT unnest(emojiSkinLabel) as Label, unnest(emojiSkinCount) as Freq FROM emoji_tweet WHERE (emojiSkinCountSum>0 AND LOWER(text) LIKE '%beach%')) as T group by T.Label order by TFreq DESC;"

<h1> Search surounding text. Search in surrounding sentence </h1>

In [ ]:
"SELECT label,SUM(C) as Freq  from (SELECT unnest(emojiLabel) as label, unnest(emojiCount) as C, unnest(prev_sentence) as prev, unnest(next_sentence) as next FROM emoji_tweet) ST WHERE (LOWER(prev) LIKE '%turtle%' OR LOWER(next) LIKE '%turle%') group by label order by Freq DESC limit 10;"
#use emojiLabel[1] for most freqent search

In [88]:
cur.execute("SELECT T.Label,SUM(T.Freq) as TFreq From (SELECT unnest(ARRAY[emojiLabel[1],emojistrLabel[1],emojiPatternLabel[1]]) as Label, unnest(ARRAY[emojiCount[1],emojistrCount[1],emojiPatternCount[1]]) as Freq FROM emoji_tweet WHERE (LOWER(text) LIKE '%hey%')) as T group by T.Label order by TFreq DESC limit 15;")

In [89]:
aa=cur.fetchall()
aa

[(None, None),
 ('\xf0\x9f\x98\x82', 2480L),
 ('\xf0\x9f\x98\xad', 737L),
 ('\xf0\x9f\x98\x8a', 576L),
 ('\xf0\x9f\x92\x80', 411L),
 ('\xf0\x9f\x98\x82\xf0\x9f\x98\x82', 402L),
 ('\xf0\x9f\x98\x8d', 393L),
 ('\xf0\x9f\x99\x84', 393L),
 ('\xf0\x9f\x98\x82\xf0\x9f\x98\x82\xf0\x9f\x98\x82', 337L),
 ('\xf0\x9f\x92\xaf', 273L),
 ('\xe2\x9d\xa4\xef\xb8\x8f', 268L),
 ('\xe2\x9c\xa8', 249L),
 ('\xf0\x9f\x92\x95', 222L),
 ('\xf0\x9f\xa4\x94', 220L),
 ('\xf0\x9f\x98\xa9', 217L)]

In [107]:
cur.execute("SELECT tweet_id from has_emoji WHERE MOD(tweet_id,2)=%s order by tweet_id DESC limit 1;",[0])

In [108]:
cur.fetchone()

(3660932,)

In [607]:
cur.execute("SELECT text FROM emoji_tweet;")
cur.fetchone()

('Beyond grateful \xf0\x9f\x98\xa9',)

In [750]:
original_text=u'@augusto_rossi1 👋 😂😂😂 😂😂kk 🇺🇸 kkk😍 k 🖐🖐 🖐 😍paliaço ✋🏾✋🇺🇸💫 m \n @augusto_rossi1 😂😂😂 😂😂kkkkk😍 k😍paliaço ✋🏾🇺🇸💫'

In [497]:
original_text=u'😂😂😂 😍😍😍😍😍😍'

In [80]:
original_text='trickfollowfas: Retweet para ganhar seguidores, siga quem retweetou e siga de volta quem te seguiu\
@JustinMarkell I can only imagine. https://t.co/aQ6ziG7tGp trickfollowfas: Retweet para ganhar seguidores, siga quem retweetou e siga de volta quem te seguiu\
@JustinMarkell I can only imagine. https://t.co/aQ6ziG7tGp'

In [503]:
print(emoji_split(original_text))

@augusto_rossi1 😂😂😂 😂😂 kkkkk 😍 k 😍 paliaço ✋🏾✋🇺🇸💫 m
@augusto_rossi1 😂😂😂 😂😂 kkkkk 😍 k 😍 paliaço ✋🏾🇺🇸💫


In [51]:
start=datetime.datetime.utcnow()
text=emoji_split(original_text)
#emjText=np.array([(emcode, len(re.findall(emcode,text))) for emcode in emj_codes])
datetime.datetime.utcnow()-start



datetime.timedelta(0, 0, 93720)

In [278]:
[' '+label+' ' for label in emojiLabel]

[u' \u270b ',
 u' \u270b\U0001f3fe ',
 u' \U0001f1fa\U0001f1f8 ',
 u' \U0001f4ab ',
 u' \U0001f602 ',
 u' \U0001f60d ']

In [275]:
text[text.index(u' \u270b '):]

u' \u270b \U0001f1fa\U0001f1f8 \U0001f4ab m @augusto_rossi1 \U0001f602 \U0001f602 \U0001f602 \U0001f602 \U0001f602 kkkkk \U0001f60d k \U0001f60d palia\xe7o \u270b\U0001f3fe \U0001f1fa\U0001f1f8 \U0001f4ab'

In [91]:
#function to quickly scan for has emoji
start=datetime.datetime.utcnow()
has_emoji=False
for emcode in emj_codes:
    if (len(re.findall(emcode,original_text))) > 0:
        has_emoji=True
        break
datetime.datetime.utcnow()-start

datetime.timedelta(0, 0, 81217)

In [235]:
print(original_text)

@augusto_rossi1 😂😂😂 😂😂kkkkk😍 k😍paliaço ✋🏾✋🇺🇸💫 m @augusto_rossi1 😂😂😂 😂😂kkkkk😍 k😍paliaço ✋🏾🇺🇸💫


In [684]:
start=datetime.datetime.utcnow()

has_emoji=False
text=emoji_split_all(original_text)
emojiLabel=np.intersect1d(text.split(),emj_codes,assume_unique=False)
datetime.datetime.utcnow()-start

datetime.timedelta(0, 0, 191877)

In [685]:
for val in emojiLabel:
    print val

✋
✋🏾
🇺🇸
💫
😂
😍


In [447]:
a=u'😂'
len(a.split())
re.findall("[a-z]",a) 

[]

In [257]:
text.split().count(emcode)# count occurences in the set

4

In [751]:
start=datetime.datetime.utcnow()
text=emoji_split_all(original_text) #split text, then find labels
emojiLabel=np.intersect1d(text.split(),emj_codes,assume_unique=False)
emjText=np.array([(emcode, text.split().count(emcode)) for emcode in emojiLabel\
        if text.split().count(emcode)>0])
datetime.datetime.utcnow()-start

datetime.timedelta(0, 0, 184228)

In [417]:
#function used currently (now replaced) this double counts emojis that can have skin tones
start=datetime.datetime.utcnow()
text=emoji_split(original_text)
emjText=np.array([(emcode, len(re.findall(emcode,text))) for emcode in emj_codes\
        if re.findall(emcode,text)!=[]])
datetime.datetime.utcnow()-start

datetime.timedelta(0, 0, 122423)

In [756]:
print(original_text)

@augusto_rossi1 👋 😂😂😂 😂😂kk 🇺🇸 kkk😍 k 🖐🖐 🖐 😍paliaço ✋🏾✋🇺🇸💫 m 
 @augusto_rossi1 😂😂😂 😂😂kkkkk😍 k😍paliaço ✋🏾🇺🇸💫


In [300]:
surroundingText(text,emojiLabel)[-1]

array([u'\U0001f60d', u'kkkkk', u'palia\xe7o',
       u'@augusto_rossi1 \U0001f602 \U0001f602 \U0001f602 \U0001f602 \U0001f602 kkkkk',
       u'k \U0001f60d palia\xe7o \u270b\U0001f3fe \u270b \U0001f1fa\U0001f1f8 \U0001f4ab'], 
      dtype='<U36')

In [471]:
np.argsort(emjText[:,1].astype(int))

array([0, 1, 2, 3, 5, 4])

In [472]:
emjText[np.argsort(emjText[:, 1].astype(int))[::-1]][:,0] #sort by freqency

array([u'\U0001f602', u'\U0001f60d', u'\U0001f4ab',
       u'\U0001f1fa\U0001f1f8', u'\u270b\U0001f3fe', u'\u270b'], 
      dtype='<U4')

In [508]:
emojiLabel=emjText[np.argsort(emjText[:, 1].astype(int))[::-1]][:,0]
emojiLabel

array([u'\U0001f602', u'\U0001f60d', u'\U0001f4ab',
       u'\U0001f1fa\U0001f1f8', u'\u270b\U0001f3fe', u'\u270b'], 
      dtype='<U4')

In [753]:
#emoji skin
emjText_skin=[(emcode, len(re.findall(emcode,text))) for emcode in emj_codes_skin\
                       if (re.findall(emcode,text) !=[])]
emjText_skinYellow=np.array([(emcode, text.split().count(emcode)) for emcode in can_have_skin\
        if text.split().count(emcode)>0])
#note: findall will look for the skin codes in the double unicode singlets. text.split().count will explicitly look for emojois that could have had a skin tone but didn't
#emjText_skin=np.vstack((emjText_skin,np.array([u'🖐',sum(emjText_skinYellow[:,1].astype(int))])))

In [754]:
#add the count with yellow skin tones
emjText_skin.append((u'\U0001f590',sum(emjText_skinYellow[:,1].astype(int))))

In [755]:
emjText_skin

[(u'\U0001f3fe', 2), (u'\U0001f590', 5)]

In [713]:
#function used currently for str
start=datetime.datetime.utcnow()
text=emoji_split(original_text)
emj_str = np.array([(emj_str, int(len(emj_str)/2)) for emj_str in sum([''.join([word if word in emj_codes+[' '] \
        else 'T' for word in emoji_split_line(line).split()]).rsplit('T') for line in text.split('\n')],[]) if emj_str != ''])
datetime.datetime.utcnow()-start  

datetime.timedelta(0, 0, 236742)

In [495]:
emj_str #len does not equal 1. And if 2, not in emcodes

array([[u'\U0001f602\U0001f602\U0001f602', u'3'],
       [u'\U0001f602\U0001f602', u'2'],
       [u'\U0001f60d', u'1'],
       [u'\U0001f60d', u'1'],
       [u'\u270b\U0001f3fe\u270b\U0001f1fa\U0001f1f8\U0001f4ab', u'5'],
       [u'\U0001f602\U0001f602\U0001f602', u'3'],
       [u'\U0001f602\U0001f602', u'2'],
       [u'\U0001f60d', u'1'],
       [u'\U0001f60d', u'1'],
       [u'\u270b\U0001f3fe\U0001f1fa\U0001f1f8\U0001f4ab', u'4']], 
      dtype='<U10')

In [723]:
#original_text='nope'
start=datetime.datetime.utcnow()
text=emoji_split(original_text)
emj_str = np.array([(emj_str, int(len(emj_str)/2)) for emj_str in sum([''.join([word if word in emojiLabel.tolist()+[' '] \
        else 'T' for word in emoji_split_line(line).split()]).rsplit('T') for line in text.split('\n')],[]) if emj_str != ''])
datetime.datetime.utcnow()-start



datetime.timedelta(0, 0, 144353)

In [726]:
emj_str

array([], dtype=float64)

In [724]:
for val in emj_str:
    print val[0]

In [725]:
d=collections.defaultdict(lambda:0)
for key in emj_str[:,0]:
    if key not in emj_codes:#do not include length 2 emoji codes, flags, skins, etc.
        d[key]+=1
a=np.array([d.values(),d.keys(),[int(len(key)/2) for key in d.keys()]])
for val in a[1]:
    print val

IndexError: too many indices for array

In [28]:
start=datetime.datetime.utcnow()

has_emoji=False
text=emoji_split_all(original_text)
emojiLabel=np.intersect1d(text.split(),emj_codes,assume_unique=False)

if len(emojiLabel) >0:
    has_emoji=True
    ###
    text=emoji_split(original_text)
    emjText=np.array([(emcode, len(re.findall(emcode,text))) for emcode in emojiLabel])
    ####
    mostFreqWord, mostFreqWordCount = count_words(text)
    newlineCount= text.count('\n')
    #create arrays to save in SQL. Sorted by frequency
    emojiLabel=emjText[np.argsort(emjText[:, 1])[::-1]][:,0]
    emojiLabelFaceFilter= np.in1d(emojiLabel,emj_codes_face,invert=True)
    emojiCount=np.array(emjText[np.argsort(emjText[:, 1])[::-1]][:,1],dtype=int)
    emojiTypes=len(emojiCount)
    emojiCountSum=sum(emojiCount)
    surrounding_text=surroundingText(text,emojiLabel) #sorted by frequency
    prev_word=surrounding_text[:,1]
    next_word=surrounding_text[:,2]
    prev_sentence=surrounding_text[:,3]
    next_sentence=surrounding_text[:,4]

    #build array of emoji strings. Look for all strings of emojis. Seperate at non-emoji char, space and \n
    emj_str = np.array([(emj_str, int(len(emj_str)/2)) for emj_str in sum([''.join([word if word in emojiLabel.tolist()+[' '] \
        else 'T' for word in emoji_split_line(line).split()]).rsplit('T') for line in text.split('\n')],[]) if emj_str != ''])
    #analyze emoji strings, cut away length 1 emojis and call new array a:
    if len(emj_str)==0:
        emojistrLabel,emojistrCount,emojistrLen,emojistrTypes,emojistr_prev_word,emojistr_next_word,\
        emojistr_prev_sentence,emojistr_next_sentence,emojiPatternLabel,emojiPatternCount,emojiPatternLen,\
        emojiPatternTypes=\
        [],[],[],0,[],[],[],[],[],[],[],0
    else: #try to find strings, but first filter length 1 and those with length 2(with skin codes)
        d=collections.defaultdict(lambda:0)
        for key in emj_str[:,0]:
            d[key]+=1
        a=np.array([d.values(),d.keys(),[int(len(key)/2) for key in d.keys()]])
        #remove single emojis and double if skin code is included:
        skin_cut=~np.bool8(((np.int32(a[2,:]))==2) & (np.array([sum([len(re.findall(emcode,val)) for emcode in emj_codes_skin]) for val in a[1,:]])))
        multi_cut=(np.int32(a[2,:])>1) & skin_cut
        a=a[:,multi_cut]

        if len(a[0])==0:
            emojistrLabel,emojistrCount,emojistrLen,emojistrTypes,emojistr_prev_word,emojistr_next_word,\
            emojistr_prev_sentence,emojistr_next_sentence,emojiPatternLabel,emojiPatternCount,emojiPatternLen,\
            emojiPatternTypes=\
            [],[],[],0,[],[],[],[],[],[],[],0

        else:
            sort_index=a.argsort(axis=1)
            emojistrLabel=a[[1,sort_index[0]]][::-1]
            emojistrCount=np.array(a[[0,sort_index[0]]][::-1],dtype=int)
            emojistrLen=np.array(a[[2,sort_index[0]]][::-1],dtype=int)
            emojistrTypes=len(emojistrCount)
            #add emjStr CountSum
            surrounding_str_text=surroundingText(text,emojistrLabel) #sorted by frequency
            emojistr_prev_word=surrounding_str_text[:,1]
            emojistr_next_word=surrounding_str_text[:,2]
            emojistr_prev_sentence=surrounding_str_text[:,3]
            emojistr_next_sentence=surrounding_str_text[:,4]
            #find emoji str patterns
            pattern=np.array([(emcode, len(re.findall(emcode,text))) for emcode in emojistrLabel])
            emojiPatternLabel=pattern[np.argsort(pattern[:, 1])[::-1]][:,0] 
            emojiPatternCount=np.array(pattern[np.argsort(pattern[:, 1])[::-1]][:,1],dtype=int)
            emojiPatternLen=np.array([np.int32(len(val)/2) for val in emojiPatternLabel],dtype=int)
            emojiPatternTypes=len(emojiPatternCount)

    #skin tone information
    emjText_skin=np.array([(emcode, len(re.findall(emcode,text))) for emcode in emj_codes_skin\
                  if (len(re.findall(emcode,text)) > 0)])
    if len(emjText_skin)==0:
        emojiSkinLabel, emojiSkinCount,emojiSkinCountSum,emojiSkinTypes= [],[],0,0
    else:
        #create arrays to save in SQL. Sorted by frequency
        emojiSkinLabel=emjText_skin[np.argsort(emjText_skin[:, 1])[::-1]][:,0] 
        emojiSkinCount=np.array(emjText_skin[np.argsort(emjText_skin[:, 1])[::-1]][:,1],dtype=int)
        emojiSkinCountSum=sum(emojiSkinCount)
        emojiSkinTypes=len(emojiSkinCount)
        
DT=datetime.datetime.utcnow()-start
DT.microseconds/1e6

0.069101

In [289]:
for val in emojiLabel:
    print val

😍
💫
🇺🇸
✋🏾
😂


In [77]:
a=np.in1d(emojiLabel,emj_codes_face,invert=True).tolist()

[False, True, True, True, True, False]

In [ ]:
if val[0] not in (self.emj_codes_face +

In [868]:
def checkNone(val):
    return val if val else ''
def checkNoneJSON(val):
    return val if val else '{}'

#tweet data:
date= datetime.datetime.utcnow()
created_at = datetime.datetime.utcnow()
#text = 'hey'
#keep split text?
retweet_count = 1
favorite_count = 2
lang=checkNone('eng')
geo = checkNoneJSON(None)
time_zone = checkNone('NewYork')
coordinates = checkNoneJSON(None)
name = 'Attila'
user_name = 'AttilaDaHun'

insertIntoSQL(date,created_at,text,retweet_count,favorite_count,lang,geo,coordinates,time_zone,name,user_name,\
emojiLabel,emojiCount,emojiCountSum,emojiTypes,prev_word,next_word,prev_sentence,next_sentence,mostFreqWord,\
mostFreqWordCount,newlineCount,emojiSkinLabel,emojiSkinCount,emojiSkinCountSum,emojiSkinTypes,emojistrLabel,\
emojistrCount,emojistrLen,emojistrTypes,emojistr_prev_word,emojistr_next_word,emojistr_prev_sentence,\
emojistr_next_sentence,emojiPatternLabel,emojiPatternCount,emojiPatternLen,emojiPatternTypes)

#TestinsertIntoSQL(date,created_at,text,retweet_count,favorite_count,lang,geo,coordinates,time_zone,name,user_name,\
#emojiLabel,emojiCount)

In [779]:
next_sentence

['']

In [776]:
u'\udfc7\udc00'.encode('utf-8')

'\xed\xbf\x87\xed\xb0\x80'

In [259]:
cur.execute("SELECT * FROM emoji_tweet;")
for val in cur.fetchall():
    print(val)

(1, datetime.datetime(2016, 4, 18, 3, 18, 37, 843870), datetime.datetime(2016, 4, 18, 3, 18, 37), 'Beyond grateful \xf0\x9f\x98\xa9', 0, 0, 'en', {}, {}, '', 'kionna.', 'vance_kionna', ['\xf0\x9f\x98\xa9'], [1], 1, 1, ['grateful'], [''], ['Beyond grateful'], [''], 'beyond', 1, 0, [], [], 0, 0, [], [], [], 0, [], [], [], [], [], [], [], 0)
(2, datetime.datetime(2016, 4, 18, 3, 18, 38, 424079), datetime.datetime(2016, 4, 18, 3, 18, 37), 'RT @madisonc0510: @meganmarie24_ omg Megan I love you\xf0\x9f\x98\x82\xf0\x9f\x92\x97 but that all you seriously', 0, 0, 'en', {}, {}, '', 'megs \xe2\x98\x80\xef\xb8\x8f', 'meganmarie24_', ['\xf0\x9f\x92\x97', '\xf0\x9f\x98\x82'], [1, 1], 2, 2, ['you\xf0\x9f\x98\x82', 'you'], ['but', '\xf0\x9f\x92\x97'], ['@madisonc0510: @meganmarie24_ omg Megan I love you\xf0\x9f\x98\x82', '@madisonc0510: @meganmarie24_ omg Megan I love you'], ['but that all you seriously', '\xf0\x9f\x92\x97 but that all you seriously'], 'rt', 1, 0, [], [], 0, 0, [], [], [], 0, [], [], 

In [141]:
cur.execute("SELECT * FROM emoji_test;")
for vals in cur.fetchall():
    for val in vals:
        print(val)

1
2016-04-18 00:57:22.948422
2016-04-18 00:57:22.948464
@alhamxdi كانت سوده ركزت لين جفت😂
1
2
eng
None
None
NewYork
Attila
AttilaDaHun
['one', 'two']
[1, 2]
2
2016-04-18 01:05:15.729238
2016-04-18 01:05:15.729284
@alhamxdi كانت سوده ركزت لين جفت😂
1
2
eng
None
None
NewYork
Attila
AttilaDaHun
['\xf0\x9f\x98\x82']
[1]
3
2016-04-18 02:09:19.744859
2016-04-18 02:09:19.744905
@alhamxdi كانت سوده ركزت لين جفت😂
1
2
eng
None
None
NewYork
Attila
AttilaDaHun
['\xf0\x9f\x98\x82']
[]


In [138]:
_u = lambda t: t.decode('UTF-8', 'replace') if isinstance(t, str) else t
_u('\xf0\x9f\x98\x82')

u'\U0001f602'

In [108]:
cur.execute("CREATE TABLE emoji_test (id serial PRIMARY KEY,\
    date timestamp,\
    created_at timestamp,\
    text text,\
    retweet_count integer,\
    favorite_count integer,\
    lang varchar(5),\
    geo json,\
    coordinates json,\
    time_zone varchar(40),\
    name varchar(40),\
    user_name varchar(40),\
    emojiLabel text[],\
    emojiCount integer[]\
    );")

In [139]:
def TestinsertIntoSQL(date,created_at,text,retweet_count,favorite_count,lang,geo,coordinates,time_zone,name,user_name,\
    emojiLabel,emojiCount):
    cur.execute("INSERT INTO emoji_test (\
    date,\
    created_at,\
    text,\
    retweet_count,\
    favorite_count,\
    lang,\
    geo,\
    coordinates,\
    time_zone,\
    name,\
    user_name,\
    emojiLabel,\
    emojiCount\
    )\
    VALUES (\
    %s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s\
    )",(\
    date,\
    created_at,\
    text,\
    retweet_count,\
    favorite_count,\
    lang,\
    geo,\
    coordinates,\
    time_zone,\
    name,\
    user_name,\
    list(emojiLabel),\
    [list(emojiCount)]
    ))

In [137]:
list(emojiLabel)

[u'\U0001f602']

In [121]:
list(emojiCount.dtype(int))

TypeError: 'numpy.dtype' object is not callable